In [54]:
import pandas as pd
import pickle as pkl

import torch,ipdb
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data
import numpy as np
import platform
import time
import matplotlib.pyplot as plt
import torchviz
import hiddenlayer as hl
from torcheval.metrics import R2Score
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()

has_gpu = torch.cuda.is_available()
has_mps = getattr(torch,'has_mps',False)
device = "mps" if getattr(torch,'has_mps',False) \
    else "gpu" if torch.cuda.is_available() else "cpu"
print(f"PyTorch Version: {torch.__version__}")
print("GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")
!jupyter --version


In [ ]:
path = '/Users/svennomm/kohalikTree/Data/AIRSCS/wave/data_v2/processed_datasets/'

with open(path + 'katse_3_4_training_data_04_19_2023_15_29_28.pkl', 'rb') as f:
    initial_data_train, initial_data_valid, target_data_train, target_data_valid, valid_index = pkl.load(f)

with open(path + 'katse_3_4_testing_data_04_19_2023_15_29_28.pkl', 'rb') as f:
    input_data_test, target_data_test, test_data_indexes = pkl.load(f)

In [55]:
path = '/Users/svennomm/kohalikTree/Data/AIRSCS/wave/data_v2/alternative_processing/'

with open(path + 'katse_03_winx_256_d_train.pkl', 'rb') as f:
    data_1 = pkl.load(f)

initial_data_train = data_1[0]
initial_data_valid = data_1[1]
target_data_train = data_1[2]
target_data_valid = data_1[3]
valid_index = data_1[4]

with open(path + 'katse_03_winx_256_d_test.pkl', 'rb') as f:
    data_1 = pkl.load(f)


input_data_test = data_1[0]
target_data_test = data_1[1]
test_data_indexes = data_1[2]

print(np.isnan(initial_data_train).any())

False


In [17]:
X_train = initial_data_train.values
y_train = target_data_train.values

X_test = initial_data_valid.values
y_test = target_data_valid.values

X_valid = input_data_test.values
y_valid = target_data_test.values

In [56]:
X_train = initial_data_train
y_train = target_data_train

X_test = initial_data_valid
y_test = target_data_valid

X_valid = input_data_test
y_valid = target_data_test

In [57]:
print(type(X_train[3, 5]))
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size = 0.7, test_size=0.3)

<class 'numpy.float32'>


In [58]:
device = torch.device('mps')
def create_dataset(X_train, y_train):
    X, y = [], []
    nx, mx = X_train.shape
    ny, my = y_train.shape
    for i in range(0, nx):
        ax = np.transpose(X_train[i,:]).reshape([mx, 1])
        ay = np.transpose(y_train[i, :]).reshape([my, 1])
        X.append(ax)
        y.append(ay)
    return torch.tensor(X).float(), torch.tensor(y).float()

In [59]:
X_train, y_train = create_dataset(X_train, y_train)
X_test, y_test = create_dataset(X_test, y_test)
X_valid, y_valid = create_dataset(X_valid, y_valid)

X_train, y_train = X_train.to('mps'), y_train.to('mps')
X_test, y_test = X_test.to('mps'), y_test.to('mps')
X_valid, y_valid = X_valid.to('mps'), y_valid.to('mps')

In [60]:
class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=32, num_layers=2, batch_first=True)
        #self.norm1 = torch.nn.LayerNorm([1, 16,16,1])
        #self.lstm_1 = nn.LSTM(32, 32, 1)
        #self.lstm_2 = nn.LSTM(8, 8, 1)
        #self.lstm_3 = nn.LSTM(8, 8, 1)
        #self.lstm_4 = nn.LSTM(8, 8, 1)
        self.linear = nn.Linear(32, 1)


    def forward(self, x):
        x, _ = self.lstm(x)
        #x, _ = self.norm1(x)
        #x, _ = self.lstm_1(x)
        #x, _ = self.lstm_2(x)
        #x, _ = self.lstm_3(x)
        #x, _ = self.lstm_4(x)
        x = self.linear(x)
        return x

In [61]:
model = AirModel()
model.to('mps')
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
loss_fn = nn.MSELoss()
print(type(X_train), torch.Tensor.size(X_train), type(y_train), torch.Tensor.size(y_train))
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=200)
metric = R2Score()

rmse_train = []
rmse_test = []

<class 'torch.Tensor'> torch.Size([1948, 69, 1]) <class 'torch.Tensor'> torch.Size([1948, 69, 1])


In [22]:
n_epochs = 50

for epoch in range(n_epochs):
    start = time.time()
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 100 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        #train_rmse = np.sqrt(loss_fn(y_pred, y_train))
        train_rmse = torch.sqrt(loss_fn(y_pred, y_train))
        train_rmse = train_rmse.detach().cpu().numpy()
        rmse_train.append(train_rmse)
        y_pred = model(X_test)
        #test_rmse = np.sqrt(loss_fn(y_prd, y_test))
        test_rmse = torch.sqrt(loss_fn(y_pred, y_test))
        test_rmse = test_rmse.detach().cpu().numpy()
        rmse_test.append(test_rmse)

        #metric.update(y_pred, y_train)
        #train_r2 = metric.compute()
        #print(train_r2)

        #metric.update(y_pred, y_test)
        #test_r2 = metric.compute()
        #print(test_r2)

    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))
    end = time.time()
    print(end - start)